In [2]:
import eegPinelineDesign
import numpy as np
import random
import mne
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
import re
import json
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA,FastICA
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.fftpack import fft,ifft
import math
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from scipy.signal import spectrogram,find_peaks_cwt
from mne.preprocessing.ica import ICA
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize

C:\Anaconda3\lib\site-packages\pandas\__init__.py:7: DeprecationWarning: bad escape \s
  from pandas import hashtable, tslib, lib
C:\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
C:\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)


In [3]:
eegPinelineDesign.change_file_directory('C:/Users/ning/Downloads/training set')
EDFfiles, Annotationfiles = eegPinelineDesign.split_type_of_files()
file_to_read, fileName = eegPinelineDesign.pick_sample_file(EDFfiles,n=4)
print(fileName)

['64chlocs.elp', 'label_extraction.npy', 'label_extraction.pkl', 's5d2_final.edf', 's5d2_final_annotations.txt', 's6n2_final.edf', 's6n2_final_annotations.txt', 'Sleep Stage Scoring Criteria.docx', 'suj10_d1final.edf', 'suj10_d1final_annotations.txt', 'suj10_d2final.edf', 'suj10_d2final_annotations.txt', 'suj13_l2nap_day2 edited.edf', 'suj13_l2nap_day2 edited1.edf', 'suj13_l2nap_day2 edited1_annotations.txt', 'suj13_l2nap_day2 edited_annotations.txt', 'suj13_l2nap_day2 edited_C3.txt', 'suj13_l2nap_day2 edited_C4.txt', 'suj13_l2nap_day2 edited_F3.txt', 'suj13_l2nap_day2 edited_F4.txt', 'suj13_l2nap_day2 edited_O1.txt', 'suj13_l2nap_day2 edited_O2.txt', 'suj5_d1final.edf', 'suj5_d1final_annotations.txt', 'suj6_d1final.edf', 'suj6_d1final_annotations.txt', 'suj8_d1final.edf', 'suj8_d1final_annotations.txt', 'suj8_d2final.edf', 'suj8_d2final_annotations.txt', 'suj9_d1final.edf', 'suj9_d1final_annotations.txt', 'suj9_d2final.edf', 'suj9_d2final_annotations.txt', 'testing-montage-2.mtg', 'Tr

In [3]:
labelFind = eegPinelineDesign.re.compile('spindle',eegPinelineDesign.re.IGNORECASE)
result={}
for names in ['F3','F4','C3','C4','O1','O2']:
    result[names]=[]
    result=pd.DataFrame(result)
frame={}
for sample_number in range(len(EDFfiles)):
    
    file_to_read,fileName = eegPinelineDesign.pick_sample_file(EDFfiles,n=sample_number)
    print(fileName)
    
    
    channelList = ['F3','F4','C3','C4','O1','O2','ROC','LOC']
    raw = eegPinelineDesign.load_data(file_to_read,channelList,low_frequency=5,high_frequency=50)
    raw.pick_channels(['F3','F4','C3','C4','O1','O2'])
    
    '''going through the grading system to all examples, and classifier the example as good examples or bad examples.'''
    
    annotation_to_read = [x for x in Annotationfiles if fileName in x]
    file = eegPinelineDesign.pd.read_csv(annotation_to_read[0])
    spindles=[]
    for row in file.iterrows():
        currentEvent = row[1][-1]
        if labelFind.search(currentEvent):
            spindles.append(row[1][0])
    windowsize=2;startSize=0.1;sample_ind={};
    for idx,names in enumerate(['F3','F4','C3','C4','O1','O2']):
        sample_ind[names]=[]
        
        
        for sample in range(len(spindles)):

            startPiont = spindles[sample]-startSize*windowsize;endPoint = spindles[sample]+(1-startSize)*windowsize
            start,stop=raw.time_as_index([startPiont,endPoint])
            tempsegment,timespan = raw[idx,start:stop]
            centerxval = timespan[np.argmax(abs(tempsegment[0,:]))]
            startPoint=centerxval-windowsize/2;endPoint=centerxval+windowsize/2
            start,stop=raw.time_as_index([startPoint,endPoint])
            segment,time=raw[idx,start:stop]
            #data[names].append(segment)
            filter_alpha = mne.filter.band_pass_filter(segment,1000,8,12);
            RMS_alpha = np.sqrt(sum(filter_alpha[0]**2)/len(filter_alpha))
            filter_spindle = mne.filter.band_pass_filter(segment,1000,11.5,16)
            RMS_spindle = np.sqrt(sum(filter_spindle[0]**2))/len(filter_spindle)
            filter_muscle = mne.filter.band_pass_filter(segment,1000,30,40)
            RMS_muscle = np.sqrt(sum(filter_muscle[0]**2)/len(filter_muscle))
            point = 0
            if RMS_alpha/RMS_spindle < 1.2:
                point +=1
            if RMS_muscle >5*10e-4:
                point -=1
            if point ==1:
                sample_ind[names].append(1)
            elif point ==0:
                sample_ind[names].append(1/6)
            else:
                sample_ind[names].append(0)
    frame[fileName]=pd.DataFrame(sample_ind)
result = pd.concat(frame)
    #print(result)
    
result.head()
    

s5d2_final
Extracting edf Parameters from s5d2_final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3383999  =      0.000 ...  3383.999 secs...
[done]
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 2 artifacts by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    5, 6, 0, 0, 7
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 4 ICA components
Inverse transforming to PCA space
Reconstructing sensor space signals from 8 PCA components
Band-pass filtering from 5 - 50 Hz
s6n2_final
Extracting edf Parameters from s6n2_final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 2073999  =      0.000 ...  2073.999 secs...
[done]
Ready.
Fitting ICA t

C:\Anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 7.5Hz is only 9.0dB.
  '%0.1fdB.' % (att_freq, att_db))
C:\Anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 16.5Hz is only 8.9dB.
  '%0.1fdB.' % (att_freq, att_db))
C:\Anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 29.5Hz is only 9.1dB.
  '%0.1fdB.' % (att_freq, att_db))
C:\Anaconda3\lib\site-packages\mne\filter.py:333: UserWarning: Attenuation at stop frequency 16.5Hz is only 9.0dB.
  '%0.1fdB.' % (att_freq, att_db))


suj8_d1final
Extracting edf Parameters from suj8_d1final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3582999  =      0.000 ...  3582.999 secs...
[done]
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 2 artifacts by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    2, 5, 7, 7, 3
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 4 ICA components
Inverse transforming to PCA space
Reconstructing sensor space signals from 8 PCA components
Band-pass filtering from 5 - 50 Hz
suj8_d2final
Extracting edf Parameters from suj8_d2final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3563999  =      0.000 ...  3563.999 secs...
[done]
Ready.
Fitti

C3        C4        F3        F4        O1        O2
s5d2_final 0  0.166667  0.166667  0.166667  0.166667  0.166667  0.166667
           1  1.000000  1.000000  0.166667  0.166667  0.166667  0.166667
           2  1.000000  0.166667  0.166667  1.000000  1.000000  0.166667
           3  0.166667  0.166667  0.166667  0.166667  0.166667  0.166667
           4  0.166667  1.000000  1.000000  0.166667  1.000000  1.000000

In [4]:
result['sum']=result.sum(axis=1)

In [ ]:
def normalized(x):
    #print(x.shape)
    normalized_x = (x-np.mean(x))/np.linalg.norm((x-np.mean(x)))
    return normalized_x
filter_result = {}

Dict_of_annotation = {}
data = {}
#data=pd.DataFrame(data,columns=['F3','F4','C3','C4','O1','O2','label'])
plt.figure(9)
for sample_number in range(len(EDFfiles[:1])):
    
    file_to_read,fileName = eegPinelineDesign.pick_sample_file(EDFfiles,n=sample_number)
    print(fileName)
    channelList = ['F3','F4','C3','C4','O1','O2','ROC','LOC']
    raw = eegPinelineDesign.load_data(file_to_read,channelList,low_frequency=8,high_frequency=16)
    raw.pick_channels(['F3','F4','C3','C4','O1','O2'])
    '''Here I take a subset of the dataframe by its file name'''
    sub_frame = result.ix[fileName]
    '''Here I take a sub-subset of the subset by the rows have a sum greater than 5 -- aka perfect example'''
    sub_group = sub_frame[sub_frame['sum'] >= 5]
    '''Reload the annotations to extract these examples'''
    annotation_to_read = [x for x in Annotationfiles if fileName in x]
    file = eegPinelineDesign.pd.read_csv(annotation_to_read[0])
    spindles=[]
    for row in file.iterrows():
        currentEvent = row[1][-1]
        if labelFind.search(currentEvent):
            spindles.append(row[1][0])
    #'''Going through the annotations and determine how consistent the examples are acrosss 6 six channels,
    #if the example matches some criterion, it is stored by its normalized version. 
    # At the same time, I will throw in the same amount of non events. 
       
    # Still thinking about should or should not throw in k-complex since they are like 1/10 size of spindles'''

    #'''reload annotations'''
    labels = {};tempdata={}
    for names in ['F3','F4','C3','C4','O1','O2']:
        labels[names]=[]
        tempdata[names]=[]
    tempdata['label']=[]
    #'''iterate rows of the sub-subset of the data frame, and throw in relevent information one by one'''
    
    for rows in sub_group.iterrows():
        #print(rows[0])
        annotation_index = rows[0];
        tempdata['label'].append(1)#odd position append spindle events
        tempdata['label'].append(0)#even position append non-events
        windowsize=2;startSize=0.1
        startPoint=spindles[annotation_index]-startSize*windowsize
        endPoint=spindles[annotation_index]+(1-startSize)*windowsize
        start,stop=raw.time_as_index([startPoint,endPoint])
        for idx,names in enumerate(['F3','F4','C3','C4','O1','O2']):
            tempsegment,timespan=raw[idx,start:stop]
            centerxval=timespan[np.argmax(abs(tempsegment))]
            startPoint=centerxval-windowsize/2
            endPoint=centerxval+windowsize/2
            start,stop=raw.time_as_index([startPoint,endPoint])
            segment,time = raw[idx,start:stop]
            #filter_segment = mne.filter.band_pass_filter(segment,1000,10,16)
            segment = (segment - segment.mean())/np.linalg.norm((segment - segment.mean()))
            #print(segment.shape)
            tempdata[names].append(segment)
            plt.plot(segment,'b',alpha=0.3)
            # put in non event, in this case, I have the same number of target events and noise events. 
            nonEvent_time_index = spindles[10] 
            while (np.array(spindles) < nonEvent_time_index + 2).all() or (np.array(spindles) > nonEvent_time_index - 2).all():
                nonEvent_time_index = np.random.choice(np.arange(0,raw.last_samp/1000))
            startPoint=nonEvent_time_index-windowsize/2;
            endPoint=nonEvent_time_index+windowsize/2
            start,stop=raw.time_as_index([startPoint,endPoint])
            segment,time=raw[idx,start:stop]
            #filter_segment = mne.filter.band_pass_filter(segment,1000,10,16)
            segment = (segment - segment.mean())/np.linalg.norm((segment - segment.mean()))
            tempdata[names].append(segment)
            plt.plot(segment,'r')
    tempdata = pd.DataFrame(tempdata)        
    data[fileName]=tempdata
    
            

In [ ]:
data

In [ ]:
resultData = data

In [ ]:
time